<a href="https://colab.research.google.com/github/FaizKhalifah/UNB-CIC-IOT-2023-Dataset-classification/blob/main/Proyek_Pak_Mahendra_Data_Random_Forrest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Dataset

In [ ]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
! kaggle datasets download -d madhavmalhotra/unb-cic-iot-dataset

Dataset URL: https://www.kaggle.com/datasets/madhavmalhotra/unb-cic-iot-dataset
License(s): other
100% 2.77G/2.77G [00:28<00:00, 68.0MB/s]
100% 2.77G/2.77G [00:28<00:00, 104MB/s] 


In [ ]:
! chmod 600 /content/kaggle.json

In [ ]:
! unzip unb-cic-iot-dataset -d data_siap_pakai

Archive:  unb-cic-iot-dataset.zip
  inflating: data_siap_pakai/wataiData/README - README.pdf  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
  inflating: data_siap_pakai/wataiData/csv/CICIoT2023/part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv  
 

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
rootFolder = 'data_siap_pakai/'
dataFolder = rootFolder+"wataiData/"
csvFolder = dataFolder+"csv/"
datasetFolder = csvFolder+"datasets/"

## Membaca Dataset

In [ ]:
df_sets = [k for k in os.listdir(datasetFolder) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets)*.8)]
test_sets = df_sets[int(len(df_sets)*.8):]

In [ ]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

## Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
scaler = StandardScaler()

In [ ]:
for train_set in tqdm(training_sets):
    scaler.fit(pd.read_csv(datasetFolder + train_set)[X_columns])

100%|██████████| 135/135 [05:16<00:00,  2.34s/it]


## Training

In [ ]:
# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the training data
for train_set in tqdm(training_sets):
    df_train = pd.read_csv(datasetFolder + train_set)
    X_train = scaler.transform(df_train[X_columns])
    y_train = df_train[y_column]  # assuming 'class' is the target column name
    rf_model.fit(X_train, y_train)


100%|██████████| 135/135 [2:19:11<00:00, 61.87s/it]


## Evaluation

In [ ]:
# Evaluate the model using the test data
all_predictions = []
all_true_labels = []

for test_set in tqdm(test_sets):
    df_test = pd.read_csv(datasetFolder + test_set)
    X_test = scaler.transform(df_test[X_columns])
    y_test = df_test[y_column]  # assuming 'class' is the target column name
    predictions = rf_model.predict(X_test)

    all_predictions.extend(predictions)
    all_true_labels.extend(y_test)

100%|██████████| 34/34 [07:50<00:00, 13.84s/it]


In [ ]:
# Calculate accuracy and other metrics
accuracy = accuracy_score(all_true_labels, all_predictions)
report = classification_report(all_true_labels, all_predictions)

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9915748894045266
Classification Report:
                         precision    recall  f1-score   support

       Backdoor_Malware       0.50      0.00      0.01       728
          BenignTraffic       0.85      0.98      0.91    243322
       BrowserHijacking       0.57      0.08      0.13      1250
       CommandInjection       0.67      0.07      0.13      1154
 DDoS-ACK_Fragmentation       0.99      0.99      0.99     62723
        DDoS-HTTP_Flood       0.97      0.88      0.92      6450
        DDoS-ICMP_Flood       1.00      1.00      1.00   1594776
DDoS-ICMP_Fragmentation       0.99      0.99      0.99    100723
      DDoS-PSHACK_Flood       1.00      1.00      1.00    907911
       DDoS-RSTFINFlood       1.00      1.00      1.00    896126
         DDoS-SYN_Flood       1.00      1.00      1.00    900820
         DDoS-SlowLoris       0.76      0.94      0.84      5278
DDoS-SynonymousIP_Flood       1.00      1.00      1.00    795606
         DDoS-TCP_Flood       1.00   